In [1]:
#import findspark
#findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession , Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
import sys , os
!{sys.executable} -m pip install beautifulsoup4

In [4]:
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [5]:
os.environ["HADOOP_HOME"] = "C:/Spark/spark-3.5.5-bin-hadoop3"  
os.environ["HADOOP_OPTS"] = "-Djava.library.path=C:/Spark/spark-3.5.5-bin-hadoop3/bin"

In [ ]:
\

## Reading SilverDataset

In [8]:
answers_df = spark.read.parquet("SilverDataset/Answers")
badge_desc_dim_df = spark.read.parquet("SilverDataset/Badge_Desc_Dim")
badges_fact_df = spark.read.parquet("SilverDataset/Badges_Fact")
comments_df = spark.read.parquet("SilverDataset/Comments")
questions_df = spark.read.parquet("SilverDataset/Questions")
vote_types_dim_df = spark.read.parquet("SilverDataset/Vote_Types_Dim")
votes_fact_df = spark.read.parquet("SilverDataset/Votes_Fact")

In [7]:
Tags_df = spark.read.parquet("SilverDataSet/Tags")
Users_df = spark.read.parquet("SilverDataSet/Users")

In [9]:
Date_Dim_df = spark.read.parquet("SilverDataSet/Date_Dim")

In [11]:
# This is a gold layer table
Answers_Desc_Dim_df = spark.read.parquet("GoldDataSet/Answers_Desc_Dim")

## Making Conformed Dims 

In [10]:
questions_df.show()

+----------+-----------+------------+----------------+----------------+--------------------+--------------------+--------------------+-----+---------+-----------+------------+
|QuestionId|OwnerUserId|CreationDate|LastActivityDate|AcceptedAnswerId|                Body|               Title|                Tags|Score|ViewCount|AnswerCount|CommentCount|
+----------+-----------+------------+----------------+----------------+--------------------+--------------------+--------------------+-----+---------+-----------+------------+
|     95868|         -2|  2014-05-01|      2014-05-01|           95962|my attempt at mak...|Least Squares Fit...|[self-study, p-va...|    1|      135|          1|          19|
|    146732|         -2|  2015-04-16|      2015-04-17|          146762|i have some troub...|Variance reductio...|[self-study, vari...|    7|     1733|          1|           5|
|    151163|         -2|  2015-05-07|      2022-05-25|          151432|implement an esti...|Monte Carlo integ...|[self-s

In [11]:
questions_df.printSchema()

root
 |-- QuestionId: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- LastActivityDate: date (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)



**Removing nulls if exists in Tags array**

In [12]:
questions_filter_Tags_Nulls_df = questions_df.withColumn("Tags", expr("filter(Tags, tag -> tag IS NOT NULL)"))

In [13]:
questions_filter_Tags_Nulls_df.printSchema()            ####

root
 |-- QuestionId: integer (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- CreationDate: date (nullable = true)
 |-- LastActivityDate: date (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- CommentCount: integer (nullable = true)



In [14]:
questions_filter_Tags_Nulls_df.selectExpr("filter(Tags, tag -> tag IS NULL) as NullTags").show(truncate=False)      ###


+--------+
|NullTags|
+--------+
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
|[]      |
+--------+
only showing top 20 rows



## Question Desc Dim 

In [15]:
Question_Desc_Dim_df = questions_filter_Tags_Nulls_df.select('QuestionId','OwnerUserId','CreationDate','Body','Title')

In [16]:
Question_Desc_Dim_df.show()

+----------+-----------+------------+--------------------+--------------------+
|QuestionId|OwnerUserId|CreationDate|                Body|               Title|
+----------+-----------+------------+--------------------+--------------------+
|     95868|         -2|  2014-05-01|my attempt at mak...|Least Squares Fit...|
|    146732|         -2|  2015-04-16|i have some troub...|Variance reductio...|
|    151163|         -2|  2015-05-07|implement an esti...|Monte Carlo integ...|
|    252960|         -2|  2016-12-23|let b the lag ope...|Forecast of $(1-0...|
|    300402|         -2|  2017-08-29|let xsim npmusigm...|Hypothesis test a...|
|    156836|         -2|  2015-06-13|let x1xn random s...|Likelihood ratio ...|
|    158971|         -2|  2015-06-27|let x1xn a random...|Karlin-Rubin theo...|
|    222915|         -2|  2016-07-09|let x1 x2 cdots x...|Expectation of $\...|
|    152927|         -2|  2015-05-19|let x1ldotsxn be ...|UMVUE of $p^4$ wh...|
|    215770|         -2|  2016-06-01|let

In [17]:
Question_Desc_Dim_df = Question_Desc_Dim_df.orderBy('CreationDate')

In [18]:
Question_Desc_Dim_df.show()                                         ######

+----------+-----------+------------+--------------------+--------------------+
|QuestionId|OwnerUserId|CreationDate|                Body|               Title|
+----------+-----------+------------+--------------------+--------------------+
|     16198|     112726|  2009-02-02|what do you call ...|What do you call ...|
|     16207|         -2|  2009-04-12|we have a daemon ...|Correcting for ou...|
|     16214|         -2|  2009-12-05|i am trying to vi...|Algorithms to eli...|
|     16046|       1260|  2010-07-10|i have data with ...|Algorithms and me...|
|        73|         22|  2010-07-19|duplicate thread ...|What R packages d...|
|       103|          5|  2010-07-19|what is the best ...|What is your favo...|
|       100|        117|  2010-07-19|id like to see th...|What's the purpos...|
|       109|         25|  2010-07-19|following oneway ...|Power of Holm's m...|
|        33|         69|  2010-07-19|what r packages s...|R packages for se...|
|        35|         54|  2010-07-19|i h

**Adding SK**

In [19]:
Question_Desc_Dim_df = Question_Desc_Dim_df.withColumn("Questions_Desc_SK",monotonically_increasing_id()+lit(1))

In [20]:
Question_Desc_Dim_df.show()                 ###

+----------+-----------+------------+--------------------+--------------------+-----------------+
|QuestionId|OwnerUserId|CreationDate|                Body|               Title|Questions_Desc_SK|
+----------+-----------+------------+--------------------+--------------------+-----------------+
|     16198|     112726|  2009-02-02|what do you call ...|What do you call ...|                1|
|     16207|         -2|  2009-04-12|we have a daemon ...|Correcting for ou...|                2|
|     16214|         -2|  2009-12-05|i am trying to vi...|Algorithms to eli...|                3|
|     16046|       1260|  2010-07-10|i have data with ...|Algorithms and me...|                4|
|        73|         22|  2010-07-19|duplicate thread ...|What R packages d...|                5|
|        22|         66|  2010-07-19|how would you des...|Bayesian and freq...|                6|
|        44|         68|  2010-07-19|how would you exp...|Explain data visu...|                7|
|         1|        

In [21]:
Question_Desc_Dim_df = Question_Desc_Dim_df.withColumnRenamed('QuestionId', 'Questions_Desc_BK')

In [22]:
Question_Desc_Dim_df = Question_Desc_Dim_df.select('Questions_Desc_SK','Questions_Desc_BK','OwnerUserId','CreationDate','Body','Title')

**Adding no question row**

In [ ]:
# Create the row for 'no question' with Questions_Desc_SK = -1
schema = StructType([
    StructField("Questions_Desc_SK", IntegerType(), False),
    StructField("Questions_Desc_BK", IntegerType(), True),  # Nullable
    StructField("OwnerUserId", IntegerType(), True),  # Nullable
    StructField("CreationDate", DateType(), True),  # Nullable
    StructField("Body", StringType(), True),
    StructField("Title", StringType(), True),
])
no_question_row = Row(
    Questions_Desc_SK=-1,  
    Questions_Desc_BK=None,  
    OwnerUserId=None,  
    CreationDate=None,  
    Body="No question",  
    Title="No title",  
)

# Convert to DataFrame
no_question_df = spark.createDataFrame([no_question_row],schema)

# Union the new 'no question' row with the existing DataFrame
Question_Desc_Dim_df = Question_Desc_Dim_df.union(no_question_df)

In [62]:
Question_Desc_Dim_df = Question_Desc_Dim_df.orderBy('Questions_Desc_SK')

In [63]:
Question_Desc_Dim_df.show()

+-----------------+-----------------+-----------+------------+--------------------+--------------------+
|Questions_Desc_SK|Questions_Desc_BK|OwnerUserId|CreationDate|                Body|               Title|
+-----------------+-----------------+-----------+------------+--------------------+--------------------+
|               -1|             NULL|       NULL|        NULL|         No question|            No title|
|                1|            16198|     112726|  2009-02-02|what do you call ...|What do you call ...|
|                2|            16207|         -2|  2009-04-12|we have a daemon ...|Correcting for ou...|
|                3|            16214|         -2|  2009-12-05|i am trying to vi...|Algorithms to eli...|
|                4|            16046|       1260|  2010-07-10|i have data with ...|Algorithms and me...|
|                5|               73|         22|  2010-07-19|duplicate thread ...|What R packages d...|
|                6|               22|         66|  2010

In [64]:
Question_Desc_Dim_df.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/Question_Desc_Dim")

In [25]:
Question_Desc_Dim_parquet_df = spark.read.parquet("GoldDataSet/Question_Desc_Dim")      ###

In [26]:
Question_Desc_Dim_parquet_df.show()             ###

+-----------------+-----------------+-----------+------------+--------------------+--------------------+
|Questions_Desc_SK|Questions_Desc_BK|OwnerUserId|CreationDate|                Body|               Title|
+-----------------+-----------------+-----------+------------+--------------------+--------------------+
|                1|            16198|     112726|  2009-02-02|what do you call ...|What do you call ...|
|                2|            16207|         -2|  2009-04-12|we have a daemon ...|Correcting for ou...|
|                3|            16214|         -2|  2009-12-05|i am trying to vi...|Algorithms to eli...|
|                4|            16046|       1260|  2010-07-10|i have data with ...|Algorithms and me...|
|                5|               73|         22|  2010-07-19|duplicate thread ...|What R packages d...|
|                6|               22|         66|  2010-07-19|how would you des...|Bayesian and freq...|
|                7|               44|         68|  2010

## Bridge Table Tags_Questions

In [27]:
exploded_tags_df = questions_df.select(
    "QuestionId", 
    explode("Tags").alias("Tag")
)

In [28]:
exploded_tags_df.show()

+----------+--------------------+
|QuestionId|                 Tag|
+----------+--------------------+
|     95868|          self-study|
|     95868|             p-value|
|     95868|  standard-deviation|
|     95868|       least-squares|
|     95868|     sums-of-squares|
|    146732|          self-study|
|    146732|            variance|
|    146732|          simulation|
|    146732|         monte-carlo|
|    146732|numerical-integra...|
|    151163|          self-study|
|    151163|          simulation|
|    151163|         monte-carlo|
|    151163| importance-sampling|
|    252960|         time-series|
|    252960|          self-study|
|    252960|         forecasting|
|    252960|               arima|
|    300402|         probability|
|    300402|  hypothesis-testing|
+----------+--------------------+
only showing top 20 rows



In [29]:
Qs_Tags_Bridge = exploded_tags_df.join(Question_Desc_Dim_df, exploded_tags_df.QuestionId==Question_Desc_Dim_df.Questions_Desc_BK,'left' )

In [30]:
Qs_Tags_Bridge = Qs_Tags_Bridge.select('Questions_Desc_SK','Tag').orderBy('Questions_Desc_SK')

In [31]:
Qs_Tags_Bridge.show()

+-----------------+------------------+
|Questions_Desc_SK|               Tag|
+-----------------+------------------+
|                1|       terminology|
|                1|          outliers|
|                1| types-of-averages|
|                2|          outliers|
|                2|    moving-average|
|                2|    cooks-distance|
|                3|data-visualization|
|                3|      graph-theory|
|                4|                 r|
|                4|  machine-learning|
|                4|       data-mining|
|                4| feature-selection|
|                5|                 r|
|                6|       frequentist|
|                6|          bayesian|
|                7|data-visualization|
|                7|         intuition|
|                8|          bayesian|
|                8|             prior|
|                8|       elicitation|
+-----------------+------------------+
only showing top 20 rows



In [32]:
Tags_df.show()

+---+--------------------+-----+-------------+-------+
| Id|             TagName|Count|TagDescPostId|TagDesc|
+---+--------------------+-----+-------------+-------+
|  3|         elicitation|   12|         NULL|   NULL|
|  5|         open-source|   18|         NULL|   NULL|
| 18|       multivariable|   39|         NULL|   NULL|
| 52|      transportation|   11|         NULL|   NULL|
|117|             polling|   46|         NULL|   NULL|
|177|             disease|   34|         NULL|   NULL|
|196|information-retri...|  114|         NULL|   NULL|
|197|          monitoring|   14|         NULL|   NULL|
|204|               humor|    6|         NULL|   NULL|
|270|           real-time|   52|         NULL|   NULL|
|393|         conferences|    7|         NULL|   NULL|
|394|    scale-invariance|   54|         NULL|   NULL|
|428|          topologies|   21|         NULL|   NULL|
|460|                 gpu|   13|         NULL|   NULL|
|461|  parallel-computing|   46|         NULL|   NULL|
|494|     

In [42]:
total_rows = Tags_df.count()

null_percentages = Tags_df.select([
    (sum(when(col(c).isNull(), 1).otherwise(0)) / total_rows * 100).alias(c) 
    for c in Tags_df.columns
])

In [43]:
null_percentages.show()

+---+-------+-----+------------------+------------------+
| Id|TagName|Count|     TagDescPostId|           TagDesc|
+---+-------+-----+------------------+------------------+
|0.0|    0.0|  0.0|24.107701941139638|24.107701941139638|
+---+-------+-----+------------------+------------------+



In [34]:
null_counts.show()


+---+-------+-----+-------------+-------+
| Id|TagName|Count|TagDescPostId|TagDesc|
+---+-------+-----+-------------+-------+
|  0|      0|    0|          385|    385|
+---+-------+-----+-------------+-------+



In [35]:
Qs_Tags_Bridge = Qs_Tags_Bridge.join(Tags_df, Tags_df.TagName==Qs_Tags_Bridge.Tag,'left' )

In [36]:
Qs_Tags_Bridge = Qs_Tags_Bridge.select('Questions_Desc_SK','Id','TagName')

In [38]:
Qs_Tags_Bridge = Qs_Tags_Bridge.withColumnRenamed('Id','TagID')

In [39]:
Qs_Tags_Bridge.show()

+-----------------+-----+--------------------+
|Questions_Desc_SK|TagID|             TagName|
+-----------------+-----+--------------------+
|            48540| 1483|          self-study|
|            48540|  631|          simulation|
|            48540|  449|         monte-carlo|
|            48540| 1202| importance-sampling|
|            96402|  188|         probability|
|            96402|   38|  hypothesis-testing|
|            96402| 1483|          self-study|
|            47352| 1483|          self-study|
|            47352|  147|            variance|
|            47352|  631|          simulation|
|            47352|  449|         monte-carlo|
|            47352| 1463|numerical-integra...|
|            31462| 1483|          self-study|
|            31462|   39|             p-value|
|            31462|   33|  standard-deviation|
|            31462|  160|       least-squares|
|            31462|  691|     sums-of-squares|
|            81292|   30|         time-series|
|            

In [40]:
Qs_Tags_Bridge.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/Qs_Tags_Bridge")

## Working on tags 

In [ ]:
Tags_df.show()              ###

+---+--------------------+-----+-------------+-------+
| Id|             TagName|Count|TagDescPostId|TagDesc|
+---+--------------------+-----+-------------+-------+
|  3|         elicitation|   12|         NULL|   NULL|
|  5|         open-source|   18|         NULL|   NULL|
| 18|       multivariable|   39|         NULL|   NULL|
| 52|      transportation|   11|         NULL|   NULL|
|117|             polling|   46|         NULL|   NULL|
|177|             disease|   34|         NULL|   NULL|
|196|information-retri...|  114|         NULL|   NULL|
|197|          monitoring|   14|         NULL|   NULL|
|204|               humor|    6|         NULL|   NULL|
|270|           real-time|   52|         NULL|   NULL|
|393|         conferences|    7|         NULL|   NULL|
|394|    scale-invariance|   54|         NULL|   NULL|
|428|          topologies|   21|         NULL|   NULL|
|460|                 gpu|   13|         NULL|   NULL|
|461|  parallel-computing|   46|         NULL|   NULL|
|494|     

In [ ]:
Tags_df = Tags_df.withColumnRenamed('Id','Tag_BK')
Tags_df = Tags_df.withColumnRenamed('Count','Total_Count')

In [46]:
Tags_df = Tags_df.drop('TagDescPostId')

In [ ]:
Tags_df= Tags_df.withColumn("Tag_SK",monotonically_increasing_id()+lit(1))

In [51]:
Tags_df = Tags_df.select('Tag_SK','Tag_BK','TagName','Total_Count','TagDesc')

In [52]:
Tags_df.show()              ###

+------+------+--------------------+-----------+-------+
|Tag_SK|Tag_BK|             TagName|Total_Count|TagDesc|
+------+------+--------------------+-----------+-------+
|     1|     3|         elicitation|         12|   NULL|
|     2|     5|         open-source|         18|   NULL|
|     3|    18|       multivariable|         39|   NULL|
|     4|    52|      transportation|         11|   NULL|
|     5|   117|             polling|         46|   NULL|
|     6|   177|             disease|         34|   NULL|
|     7|   196|information-retri...|        114|   NULL|
|     8|   197|          monitoring|         14|   NULL|
|     9|   204|               humor|          6|   NULL|
|    10|   270|           real-time|         52|   NULL|
|    11|   393|         conferences|          7|   NULL|
|    12|   394|    scale-invariance|         54|   NULL|
|    13|   428|          topologies|         21|   NULL|
|    14|   460|                 gpu|         13|   NULL|
|    15|   461|  parallel-compu

In [53]:
Tags_df.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/Tags_Dim")

## Adjusting Users Table 

In [66]:
Users_df.show()

+------------+-----------------+---------+----------+--------------+----------+-------+-----+
|CreationDate|      DisplayName|DownVotes|UsersId_BK|LastAccessDate|Reputation|UpVotes|Views|
+------------+-----------------+---------+----------+--------------+----------+-------+-----+
|  2017-06-11|       phantomile|        0|    164892|    2021-03-16|         1|      0|    1|
|  2017-06-11|         woofwoof|        0|    164893|    2021-10-18|       101|      0|    0|
|  2017-06-12|       user164894|        0|    164894|    2017-06-19|        21|      0|    0|
|  2017-06-12|     Parth Sharma|        0|    164895|    2021-09-14|         3|      0|    1|
|  2017-06-12|Jacques Thibodeau|        0|    164896|    2023-01-04|       101|      5|    2|
|  2017-06-12|       user164897|        0|    164897|    2017-06-19|         1|      0|    0|
|  2017-06-12|             Amin|        0|    164898|    2023-03-17|       161|      0|    1|
|  2017-06-12|      Sagar Patel|        0|    164899|    201

In [67]:
Users_df = Users_df.withColumn("UsersID_SK",monotonically_increasing_id()+lit(1)).orderBy('UsersID_SK')

In [70]:
Users_df.show()

+----------+----------+------------+-----------------+--------------+----------+-----+-------+---------+
|UsersID_SK|UsersId_BK|CreationDate|      DisplayName|LastAccessDate|Reputation|Views|UpVotes|DownVotes|
+----------+----------+------------+-----------------+--------------+----------+-----+-------+---------+
|         1|    164892|  2017-06-11|       phantomile|    2021-03-16|         1|    1|      0|        0|
|         2|    164893|  2017-06-11|         woofwoof|    2021-10-18|       101|    0|      0|        0|
|         3|    164894|  2017-06-12|       user164894|    2017-06-19|        21|    0|      0|        0|
|         4|    164895|  2017-06-12|     Parth Sharma|    2021-09-14|         3|    1|      0|        0|
|         5|    164896|  2017-06-12|Jacques Thibodeau|    2023-01-04|       101|    2|      5|        0|
|         6|    164897|  2017-06-12|       user164897|    2017-06-19|         1|    0|      0|        0|
|         7|    164898|  2017-06-12|             Amin| 

In [69]:
Users_df = Users_df.select('UsersID_SK','UsersId_BK','CreationDate','DisplayName','LastAccessDate','Reputation','Views','UpVotes','DownVotes')

In [73]:
Users_df.orderBy('UsersId_BK').show()

+----------+----------+------------+------------------+--------------+----------+-----+-------+---------+
|UsersID_SK|UsersId_BK|CreationDate|       DisplayName|LastAccessDate|Reputation|Views|UpVotes|DownVotes|
+----------+----------+------------+------------------+--------------+----------+-----+-------+---------+
|        -1|        -2|        NULL|           No User|          NULL|         0|    0|      0|        0|
|8589934593|        -1|  2010-07-19|         Community|    2010-07-19|         1| 3131|  17823|     9720|
|8589934594|         2|  2010-07-19|      Geoff Dalgas|    2019-02-08|       101|   49|      3|        0|
|8589934595|         3|  2010-07-19|      Jarrod Dixon|    2019-02-07|       101|   50|     23|        0|
|8589934596|         4|  2010-07-19|            Emmett|    2016-11-24|       101|   24|      0|        0|
|8589934597|         5|  2010-07-19|             Shane|    2022-12-07|     12151| 2101|    684|        5|
|8589934598|         6|  2010-07-19|          

**Adding the no user row**

In [72]:
schema = StructType([
    StructField("UsersID_SK", IntegerType(), False),
    StructField("UsersId_BK", IntegerType(), False),
    StructField("CreationDate", DateType(), True),
    StructField("DisplayName", StringType(), True),
    StructField("LastAccessDate", DateType(), True),
    StructField("Reputation", IntegerType(), True),
    StructField("Views", IntegerType(), True),
    StructField("UpVotes", IntegerType(), True),
    StructField("DownVotes", IntegerType(), True),
])

# Create the "No User" row
no_user_row = Row(
    UsersID_SK=-1,  
    UsersId_BK=-2,  
    CreationDate=None,  
    DisplayName="No User",  
    LastAccessDate=None,  
    Reputation=0,  
    Views=0,  
    UpVotes=0,  
    DownVotes=0
)

# Convert to DataFrame
no_user_df = spark.createDataFrame([no_user_row], schema)

# Assuming `Users_df` is the existing DataFrame
Users_df = Users_df.union(no_user_df)

In [74]:
Users_df.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/Users_Dim")

## Answers Desc Dim

In [76]:
Answers_Desc_Dim_df.orderBy('Answer_Desc_BK').show()

+--------------+--------------------+-----------+--------+--------------+
|Answer_Desc_BK|                Body|OwnerUserId|ParentId|Answer_Desc_SK|
+--------------+--------------------+-----------+--------+--------------+
|             5|the rproject http...|         23|       3|             1|
|             9|incanter is a clo...|         50|       3|             2|
|            12|see my response t...|          5|       7|             3|
|            13|machine learning ...|         23|       6|             4|
|            14|i second that jay...|         36|       3|             5|
|            15|john cook gives s...|          6|       1|             6|
|            16|two projects spri...|          8|       3|             7|
|            18|also see the uci ...|         36|       7|             8|
|            19|gapminder has a n...|         55|       7|             9|
|            20|the assumption of...|         37|       2|            10|
|            24|for doing a varie...| 

In [96]:
answers_schema = StructType([
    StructField("Answer_Desc_BK", IntegerType(), False),
    StructField("Body", StringType(), True),
    StructField("OwnerUserId", IntegerType(), True),
    StructField("ParentId", IntegerType(), True),
    StructField("Answer_Desc_SK", IntegerType(), False)
])

# Create the "no answer" row
no_answer_row = Row(
    Answer_Desc_BK = -1,   # Special business key for "no answer"
    Body = "No Answer",    # Indicate no answer text
    OwnerUserId = None,    # No owner user
    ParentId = None,       # No parent question id
    Answer_Desc_SK = -1    # Special surrogate key for "no answer"
)

# Convert the row into a DataFrame (wrap in a list)
no_answer_df = spark.createDataFrame([no_answer_row], answers_schema)

# Union the "no answer" row with your existing answers DataFrame
# Assuming your existing DataFrame is called Answers_df
Answers_Desc_Dim_df = Answers_Desc_Dim_df.union(no_answer_df)
Answers_Desc_Dim_df = Answers_Desc_Dim_df.dropDuplicates()

In [97]:
Answers_Desc_Dim_df = Answers_Desc_Dim_df.select('Answer_Desc_SK','Answer_Desc_BK','OwnerUserId','ParentId','Body')

In [105]:
Answers_Desc_Dim_df.orderBy('Answer_Desc_BK').show()

+--------------+--------------+-----------+--------+--------------------+
|Answer_Desc_SK|Answer_Desc_BK|OwnerUserId|ParentId|                Body|
+--------------+--------------+-----------+--------+--------------------+
|            -1|            -1|       NULL|    NULL|           No Answer|
|           703|          1000|        549|     951|well if you have ...|
|         42020|        100004|      13138|  100003|the formula for t...|
|         42021|        100008|      46134|   99368|maybe you should ...|
|         42022|        100009|      16663|   28984|having a quick lo...|
|         42023|        100013|       7828|   99985|since d contains ...|
|         42024|        100014|      31314|   99985|to put some more ...|
|         42025|        100015|       7828|   99983|spend lots of tim...|
|         42026|        100021|      46151|   99876|i think thats the...|
|         42027|        100022|        919|   99986|as shown at how d...|
|         42028|        100023|      2

In [99]:
Answers_Desc_Dim_df.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/Answers_Desc_Dim")

In [101]:
Date_Dim_df.show()

+----------+--------+----------+-----+----+---------+---------+----------+---------+-------+----------+
|  FullDate| DateKey|DayOfMonth|Month|Year|DayOfWeek|  DayName|WeekOfYear|MonthName|Quarter|FiscalYear|
+----------+--------+----------+-----+----+---------+---------+----------+---------+-------+----------+
|2010-01-01|20100101|         1|    1|2010|        6|   Friday|        53|  January|      1|      2010|
|2010-01-02|20100102|         2|    1|2010|        7| Saturday|        53|  January|      1|      2010|
|2010-01-03|20100103|         3|    1|2010|        1|   Sunday|        53|  January|      1|      2010|
|2010-01-04|20100104|         4|    1|2010|        2|   Monday|         1|  January|      1|      2010|
|2010-01-05|20100105|         5|    1|2010|        3|  Tuesday|         1|  January|      1|      2010|
|2010-01-06|20100106|         6|    1|2010|        4|Wednesday|         1|  January|      1|      2010|
|2010-01-07|20100107|         7|    1|2010|        5| Thursday| 

## Questions Fact Table

In [114]:
questions_df = questions_df.withColumnRenamed('CreationDate','QCreationDate')
questions_df = questions_df.withColumnRenamed('LastActivityDate','QLastActivityDate')

In [113]:
questions_df.show()         ###

+----------+-----------+------------+----------------+----------------+--------------------+--------------------+--------------------+-----+---------+-----------+------------+
|QuestionId|OwnerUserId|CreationDate|LastActivityDate|AcceptedAnswerId|                Body|               Title|                Tags|Score|ViewCount|AnswerCount|CommentCount|
+----------+-----------+------------+----------------+----------------+--------------------+--------------------+--------------------+-----+---------+-----------+------------+
|     95868|         -2|  2014-05-01|      2014-05-01|           95962|my attempt at mak...|Least Squares Fit...|[self-study, p-va...|    1|      135|          1|          19|
|    146732|         -2|  2015-04-16|      2015-04-17|          146762|i have some troub...|Variance reductio...|[self-study, vari...|    7|     1733|          1|           5|
|    151163|         -2|  2015-05-07|      2022-05-25|          151432|implement an esti...|Monte Carlo integ...|[self-s

In [115]:
Question_Fact = questions_df.join(Users_df,questions_df.OwnerUserId==Users_df.UsersId_BK,'left')\
                            .join(Date_Dim_df,questions_df.QCreationDate==Date_Dim_df.FullDate,'left')\
                            .join(Question_Desc_Dim_df,questions_df.QuestionId==Question_Desc_Dim_df.Questions_Desc_BK,'left')\
                            .join(Answers_Desc_Dim_df,questions_df.AcceptedAnswerId==Answers_Desc_Dim_df.Answer_Desc_BK,'left')\
                            .select(
                                col('Questions_Desc_SK').alias('Questions_Desc_FK'),
                                col('UsersID_SK').alias('OwnerUserID_FK'),
                                col('DateKey').alias('CreationDate_FK'),
                                col('Answer_Desc_SK').alias('AcceptedAnswerId_FK'),
                                col('Score'),
                                col('ViewCount'),
                                col('AnswerCount'),
                                col('CommentCount'),
                                col('QCreationDate'),
                                col('QLastActivityDate')                           
                                )

In [118]:
Question_Fact = Question_Fact.join(Date_Dim_df,questions_df.QLastActivityDate==Date_Dim_df.FullDate,'left')\
                             .select(
                                 '*',
                                 col('DateKey').alias('LastActivityDate_FK')
                             )

In [120]:
Question_Fact = Question_Fact.select(
                                col('Questions_Desc_FK'),
                                col('OwnerUserID_FK'),
                                col('CreationDate_FK'), 
                                col('LastActivityDate_FK'),                             
                                col('AcceptedAnswerId_FK'),
                                col('Score'),
                                col('ViewCount'),
                                col('AnswerCount'),
                                col('CommentCount'),
                                col('QCreationDate'),
                                col('QLastActivityDate')  
)

In [121]:
Question_Fact.show()

+-----------------+--------------+---------------+-------------------+-------------------+-----+---------+-----------+------------+-------------+-----------------+
|Questions_Desc_FK|OwnerUserID_FK|CreationDate_FK|LastActivityDate_FK|AcceptedAnswerId_FK|Score|ViewCount|AnswerCount|CommentCount|QCreationDate|QLastActivityDate|
+-----------------+--------------+---------------+-------------------+-------------------+-----+---------+-----------+------------+-------------+-----------------+
|             2173|            -1|       20110309|           20110310|                 -1|    1|     1731|          2|           5|   2011-03-09|       2011-03-10|
|            20186|            -1|       20130630|           20130701|              28800|    2|      623|          1|           9|   2013-06-30|       2013-07-01|
|            48540|            -1|       20150507|           20220525|              58335|    5|      601|          3|           5|   2015-05-07|       2022-05-25|
|            491

In [122]:
null_counts = Question_Fact.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(f"{c}_null_count")
    for c in Question_Fact.columns
])

null_counts.show(truncate=False)

+----------------------------+-------------------------+--------------------------+------------------------------+------------------------------+----------------+--------------------+----------------------+-----------------------+------------------------+----------------------------+
|Questions_Desc_FK_null_count|OwnerUserID_FK_null_count|CreationDate_FK_null_count|LastActivityDate_FK_null_count|AcceptedAnswerId_FK_null_count|Score_null_count|ViewCount_null_count|AnswerCount_null_count|CommentCount_null_count|QCreationDate_null_count|QLastActivityDate_null_count|
+----------------------------+-------------------------+--------------------------+------------------------------+------------------------------+----------------+--------------------+----------------------+-----------------------+------------------------+----------------------------+
|0                           |0                        |3                         |0                             |1                             |

In [127]:
Question_Fact.filter(col('CreationDate_FK').isNull()).show()


+-----------------+--------------+---------------+-------------------+-------------------+-----+---------+-----------+------------+-------------+-----------------+
|Questions_Desc_FK|OwnerUserID_FK|CreationDate_FK|LastActivityDate_FK|AcceptedAnswerId_FK|Score|ViewCount|AnswerCount|CommentCount|QCreationDate|QLastActivityDate|
+-----------------+--------------+---------------+-------------------+-------------------+-----+---------+-----------+------------+-------------+-----------------+
+-----------------+--------------+---------------+-------------------+-------------------+-----+---------+-----------+------------+-------------+-----------------+



**Dealing with nulls**

In [126]:
Question_Fact = Question_Fact.dropna()

In [128]:
Question_Fact.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/Questions_Fact")

## Votes Desc Dim 

In [21]:
vote_types_dim_df.show()

+----------+--------------------+--------------------+
|VoteTypeId|        VoteTypeName|         Description|
+----------+--------------------+--------------------+
|         1|AcceptedByOriginator|Marks an answer a...|
|         2|              Upvote|Upvote given to a...|
|         3|            Downvote|Downvote given to...|
|         4|           Offensive|Flags a post as o...|
|         5|            Favorite|Indicates a user ...|
|         6|               Close|Suggests that a q...|
|         7|              Reopen|Suggests that a c...|
|         8|         BountyStart|Indicates that a ...|
|         9|         BountyClose|Indicates that a ...|
|        10|            Deletion|Indicates that a ...|
|        11|          Undeletion|Indicates that a ...|
|        12|           Migration|Indicates that a ...|
+----------+--------------------+--------------------+



**Renaming Coloumns**

In [ ]:
vote_types_dim_df = vote_types_dim_df.withColumnRenamed('Description','VoteType_Desc')


In [14]:
votes_fact_df.show()

+----+------+----------+------------+-----------+------------+
|  Id|PostId|VoteTypeId|CreationDate|PostOwnerId|VoteTypeName|
+----+------+----------+------------+-----------+------------+
| 310|   148|         2|  2010-07-19|        130|      Upvote|
| 345|   148|         2|  2010-07-19|        130|      Upvote|
| 398|   148|         2|  2010-07-20|        130|      Upvote|
|1255|   471|         2|  2010-07-21|        168|      Upvote|
|1343|   463|         2|  2010-07-22|         90|      Upvote|
|1409|   496|         2|  2010-07-22|        190|      Upvote|
|1575|   463|         2|  2010-07-23|         90|      Upvote|
|1779|   496|         2|  2010-07-24|        190|      Upvote|
|1837|   496|         2|  2010-07-25|        190|      Upvote|
|2570|   833|         2|  2010-07-27|        495|      Upvote|
|2591|   833|         2|  2010-07-27|        495|      Upvote|
|2650|   833|         2|  2010-07-27|        495|      Upvote|
|2658|   833|         2|  2010-07-28|        495|      

In [46]:
vote_types_dim_df.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/vote_types_dim")

## Votes Fact

In [26]:
# I'm reading those for making votes fact
Question_Desc_Dim_df = spark.read.parquet("GoldDataSet/Question_Desc_Dim")
Answers_Desc_Dim_df = spark.read.parquet("GoldDataSet/Answers_Desc_Dim")
Users_df = spark.read.parquet("GoldDataSet/Users_Dim")

In [27]:
Users_df.show()

+----------+----------+------------+-----------------+--------------+----------+-----+-------+---------+
|UsersID_SK|UsersId_BK|CreationDate|      DisplayName|LastAccessDate|Reputation|Views|UpVotes|DownVotes|
+----------+----------+------------+-----------------+--------------+----------+-----+-------+---------+
|         1|    164892|  2017-06-11|       phantomile|    2021-03-16|         1|    1|      0|        0|
|         2|    164893|  2017-06-11|         woofwoof|    2021-10-18|       101|    0|      0|        0|
|         3|    164894|  2017-06-12|       user164894|    2017-06-19|        21|    0|      0|        0|
|         4|    164895|  2017-06-12|     Parth Sharma|    2021-09-14|         3|    1|      0|        0|
|         5|    164896|  2017-06-12|Jacques Thibodeau|    2023-01-04|       101|    2|      5|        0|
|         6|    164897|  2017-06-12|       user164897|    2017-06-19|         1|    0|      0|        0|
|         7|    164898|  2017-06-12|             Amin| 

In [34]:
Question_Desc_Dim_df.show()

+-----------------+-----------------+-----------+------------+--------------------+--------------------+
|Questions_Desc_SK|Questions_Desc_BK|OwnerUserId|CreationDate|                Body|               Title|
+-----------------+-----------------+-----------+------------+--------------------+--------------------+
|               -1|             NULL|       NULL|        NULL|         No question|            No title|
|                1|            16198|     112726|  2009-02-02|what do you call ...|What do you call ...|
|                2|            16207|         -2|  2009-04-12|we have a daemon ...|Correcting for ou...|
|                3|            16214|         -2|  2009-12-05|i am trying to vi...|Algorithms to eli...|
|                4|            16046|       1260|  2010-07-10|i have data with ...|Algorithms and me...|
|                5|               73|         22|  2010-07-19|duplicate thread ...|What R packages d...|
|                6|               22|         66|  2010

In [36]:
votes_fact_Final = votes_fact_df \
    .join(vote_types_dim_df, votes_fact_df.VoteTypeId == vote_types_dim_df.VoteTypeId, "left") \
    .join(Date_Dim_df, votes_fact_df.CreationDate == Date_Dim_df.FullDate, "left") \
    .join(Users_df, votes_fact_df.PostOwnerId == Users_df.UsersId_BK, "left") \
    .join(Question_Desc_Dim_df, votes_fact_df.PostId == Question_Desc_Dim_df.Questions_Desc_BK, "left") \
    .join(Answers_Desc_Dim_df, votes_fact_df.PostId == Answers_Desc_Dim_df.Answer_Desc_BK, "left") \
    .select(
        col("Id").alias("VoteId"),
        col("Questions_Desc_SK").alias("Question_Desc_FK"),
        col("Answer_Desc_SK").alias("Answer_Desc_FK"),
        col("UsersID_SK").alias("PostOwnerId_FK"),
        col("DateKey").alias("CreationDate_FK"),
        col("VoteId").alias("VoteType_FK"),
        votes_fact_df.CreationDate)

In [37]:
votes_fact_Final.show()

+------+----------------+--------------+--------------+---------------+-----------+------------+
|VoteId|Question_Desc_FK|Answer_Desc_FK|PostOwnerId_FK|CreationDate_FK|VoteType_FK|CreationDate|
+------+----------------+--------------+--------------+---------------+-----------+------------+
|   310|            NULL|            86|    8589934701|       20100719|        310|  2010-07-19|
|   345|            NULL|            86|    8589934701|       20100719|        345|  2010-07-19|
|   398|            NULL|            86|    8589934701|       20100720|        398|  2010-07-20|
|  1343|            NULL|           313|    8589934666|       20100722|       1343|  2010-07-22|
|  1575|            NULL|           313|    8589934666|       20100723|       1575|  2010-07-23|
|  1255|            NULL|           320|    8589934732|       20100721|       1255|  2010-07-21|
|  1409|            NULL|           338|    8589934752|       20100722|       1409|  2010-07-22|
|  1779|            NULL|     

In [45]:
null_counts = votes_fact_Final.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(f"{c}_null_count")
    for c in votes_fact_Final.columns
])

null_counts.show(truncate=False)

+-----------------+---------------------------+-------------------------+-------------------------+--------------------------+----------------------+-----------------------+
|VoteId_null_count|Question_Desc_FK_null_count|Answer_Desc_FK_null_count|PostOwnerId_FK_null_count|CreationDate_FK_null_count|VoteType_FK_null_count|CreationDate_null_count|
+-----------------+---------------------------+-------------------------+-------------------------+--------------------------+----------------------+-----------------------+
|0                |0                          |0                        |0                        |70                        |0                     |0                      |
+-----------------+---------------------------+-------------------------+-------------------------+--------------------------+----------------------+-----------------------+



**Handling null values** 

In [44]:
votes_fact_Final = votes_fact_Final.fillna({
    "Question_Desc_FK" : -1,
    "Answer_Desc_FK" : -1,
    "PostOwnerId_FK" : -1
})

In [50]:
votes_fact_Final = votes_fact_Final.dropna()

In [51]:
null_counts = votes_fact_Final.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(f"{c}_null_count")
    for c in votes_fact_Final.columns
])

null_counts.show(truncate=False)

+-----------------+---------------------------+-------------------------+-------------------------+--------------------------+----------------------+-----------------------+
|VoteId_null_count|Question_Desc_FK_null_count|Answer_Desc_FK_null_count|PostOwnerId_FK_null_count|CreationDate_FK_null_count|VoteType_FK_null_count|CreationDate_null_count|
+-----------------+---------------------------+-------------------------+-------------------------+--------------------------+----------------------+-----------------------+
|0                |0                          |0                        |0                        |0                         |0                     |0                      |
+-----------------+---------------------------+-------------------------+-------------------------+--------------------------+----------------------+-----------------------+



In [52]:
votes_fact_Final.coalesce(1).write.mode("overwrite").parquet("GoldDataSet/Votes_fact")

In [53]:
spark.stop()